In [239]:
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Your dataset
# Updated dataset with all columns
diseases: [
        "Influenza",
        "Dengue",
        "Malaria", 
        "Typhoid Fever", 
        "Pneumonia",
        "Appendicitis",
        "Gastroenteritis",
        "Migraine", 
        "Urinary Tract Infection (UTI)",
        "Bronchitis",
        "COVID-19", 
        "Sinusitis", 
        "Chikungunya",
        "Chickenpox",
        "Zika Virus",
        "Leptospirosis",
        "Hepatitis A",
        "Hepatitis B", 
        "Cholera",
        "Tetanus", 
        "Rabies",
        "Mumps", 
        "Measles",
        "Diphtheria",
        "Scarlet Fever",
        "Typhus",
        "Rotavirus", 
        "Norovirus",
        "Enteric Fever",
        "H1N1 (Swine Flu)"
    ]
symptoms: {
        "Influenza": ["Fever", "Cough", "Sore throat", "Muscle aches", "Fatigue"],
        "Dengue": ["High fever", "Severe headache", "Rash", "Muscle pain"],
        "Malaria": ["Chills", "Fever", "Fatigue", "Sweats"],
        "Typhoid Fever": ["Abdominal pain", "Loss of appetite", "High fever"],
        "Pneumonia": ["Chest pain", "Shortness of breath", "Fever", "Cough"],
        "Appendicitis": ["Sudden abdominal pain (lower right)", "Nausea", "Vomiting", "Low-grade fever"],
        "Gastroenteritis": ["Diarrhea", "Abdominal cramps", "Vomiting", "Low-grade fever"],
        "Migraine": ["Severe headache", "Sensitivity to light/sound", "Nausea"],
        "Urinary Tract Infection (UTI)": ["Painful urination", "Frequent urination", "Lower abdominal pain", "Cloudy urine"],
        "Bronchitis": ["Cough with mucus", "Wheezing", "Chest tightness"],
        "COVID-19": ["Fever", "Dry cough", "Loss of taste/smell", "Fatigue", "Difficulty breathing"],
        "Sinusitis": ["Facial pain", "Nasal congestion", "Headache", "Post-nasal drip"],
        "Chikungunya": ["High fever", "Joint pain", "Rash", "Headache"],
        "Chickenpox": ["Fever", "Rash (blisters)", "Itching", "Fatigue"],
        "Zika Virus": ["Fever", "Rash", "Conjunctivitis", "Joint pain"],
        "Leptospirosis": ["Fever", "Headache", "Muscle pain", "Abdominal pain"],
        "Hepatitis A": ["Fever", "Nausea", "Vomiting", "Jaundice", "Abdominal pain"],
        "Hepatitis B": ["Fatigue", "Jaundice", "Abdominal pain", "Nausea"],
        "Cholera": ["Severe diarrhea", "Vomiting", "Dehydration"],
        "Tetanus": ["Muscle stiffness", "Lockjaw", "Difficulty swallowing"],
        "Rabies": ["Fever", "Headache", "Hydrophobia", "Agitation"],
        "Mumps": ["Fever", "Swelling of salivary glands", "Headache"],
        "Measles": ["Fever", "Cough", "Rash", "Conjunctivitis"],
        "Diphtheria": ["Sore throat", "Fever", "Weakness", "Swollen glands"],
        "Scarlet Fever": ["Sore throat", "Red rash", "Fever", "Strawberry tongue"],
        "Typhus": ["Fever", "Rash", "Headache", "Muscle pain"],
        "Rotavirus": ["Severe diarrhea", "Vomiting", "Fever", "Abdominal cramps"],
        "Norovirus": ["Diarrhea", "Vomiting", "Stomach cramps"],
        "Enteric Fever": ["Abdominal pain", "High fever", "Diarrhea", "Weakness"],
        "H1N1 (Swine Flu)": ["Fever", "Cough", "Sore throat", "Body aches"]
    }
Medicines: {
        "Influenza": ["Paracetamol 500mg", "Ibuprofen 400mg", "Oseltamivir 75mg"],
        "Dengue": ["Acetaminophen 500mg", "ORS (Oral Rehydration Solution)"],
        "Malaria": ["Artemisinin-based Combination Therapy (ACT)"],
        "Typhoid Fever": ["Ciprofloxacin 500mg", "Azithromycin 250mg"],
        "Pneumonia": ["Amoxicillin 500mg", "Doxycycline 100mg"],
        "Appendicitis": ["Painkillers (e.g., Ibuprofen 200mg)", "Antibiotics (e.g., Ceftriaxone 1g)"],
        "Gastroenteritis": ["ORS (Oral Rehydration Solution)", "Loperamide 2mg", "Zinc 20mg"],
        "Migraine": ["Sumatriptan 50mg", "Ibuprofen 400mg"],
        "Urinary Tract Infection (UTI)": ["Nitrofurantoin 100mg", "Ciprofloxacin 500mg"],
        "Bronchitis": ["Bronchodilators (e.g., Salbutamol 2.5mg)", "Cough Suppressants (e.g., Dextromethorphan 10mg)"],
        "COVID-19": ["Paracetamol 500mg", "Favipiravir 200mg"],
        "Sinusitis": ["Decongestants (e.g., Pseudoephedrine 60mg)", "Antihistamines (e.g., Loratadine 10mg)"],
        "Chikungunya": ["Paracetamol 500mg", "Ibuprofen 200mg"],
        "Chickenpox": ["Calamine Lotion", "Antihistamines (e.g., Diphenhydramine 25mg)"],
        "Zika Virus": ["Paracetamol 500mg", "Fluids"],
        "Leptospirosis": ["Doxycycline 100mg", "Penicillin 500mg"],
        "Hepatitis A": ["Supportive care", "Hydration"],
        "Hepatitis B": ["Tenofovir 300mg", "Entecavir 0.5mg"],
        "Cholera": ["Rehydration (e.g., ORS)", "Antibiotics (e.g., Doxycycline 300mg)"],
        "Tetanus": ["Tetanus vaccination", "Painkillers (e.g., Ibuprofen 400mg)"],
        "Rabies": ["Post-exposure prophylaxis (PEP) with Rabies Vaccine"],
        "Mumps": ["Supportive care", "Pain relief (e.g., Paracetamol 500mg)"],
        "Measles": ["Vitamin A 100,000 IU (for children)", "Supportive care"],
        "Diphtheria": ["Penicillin 250mg", "Diphtheria Toxoid Vaccine"],
        "Scarlet Fever": ["Penicillin 500mg", "Amoxicillin 500mg"],
        "Typhus": ["Doxycycline 100mg", "Chloramphenicol 500mg"],
        "Rotavirus": ["ORS (Oral Rehydration Solution)", "Zinc 20mg"],
        "Norovirus": ["Fluids", "Antiemetics (e.g., Ondansetron 4mg)"],
        "Enteric Fever": ["Ciprofloxacin 500mg", "Azithromycin 250mg"],
        "H1N1 (Swine Flu)": ["Oseltamivir 75mg", "Paracetamol 500mg"]
    }

Description:{
        "Influenza": ["The disease Influenza causes various symptoms and can be diagnosed through specific tests."],
        
        "Dengue": ["Dengue causes a high fever with severe headache and rash."],
        
        "Malaria": ["Malaria involves chills, fever, and sweats."],
        
        "Typhoid Fever":  ["Typhoid is a bacterial infection that requires antibiotics."],
        
        "Pneumonia":["Pneumonia causes breathing difficulties and chest pain."],
        
        "Appendicitis": ["Appendicitis needs immediate surgery to avoid complications."],
        
        "Gastroenteritis": ["Gastroenteritis involves inflammation of the stomach and intestines."],
        
        "Migraine":  ["Migraines are severe headaches often accompanied by nausea."],
        
        "Urinary Tract Infection (UTI)":  ["UTI is characterized by painful urination and frequent trips to the bathroom."],
        
        "Bronchitis": ["Bronchitis causes persistent coughing and mucus production."],
        
        "COVID-19":  ["COVID-19 is a respiratory disease with diverse symptoms."],
        
        "Sinusitis": ["Sinusitis affects nasal passages and causes severe headaches."],
        
        "Chikungunya":  ["Chikungunya causes joint pain and fever."],
        
        "Chickenpox":["Chickenpox is a contagious rash illness."],
        
        "Zika Virus":  ["Zika Virus can cause birth defects in pregnant women."],
        
        "Leptospirosis": ["Leptospirosis is a bacterial infection that spreads through animal urine."],
        
        "Hepatitis A":  ["Hepatitis A is a liver infection caused by contaminated food or water."],
        
        "Hepatitis B":  ["Hepatitis B is a liver infection spread through body fluids."],
        
        "Cholera":  ["Cholera is caused by contaminated water and causes severe dehydration."],
        
        "Tetanus": ["Tetanus affects the nervous system and causes muscle stiffness."],
        
        "Rabies":  ["Rabies is a viral infection often fatal once symptoms appear."],
        
        "Mumps":  ["Mumps is a viral infection causing swollen salivary glands."],
        
        "Measles":  ["Measles causes a characteristic rash and fever."],
        
        "Diphtheria": ["Diphtheria affects the throat and nose."],
        
        "Scarlet Fever": ["Scarlet Fever involves a bright red rash."],
        
        "Typhus":  ["Typhus is transmitted by lice and fleas."],
        
        "Rotavirus":  ["Rotavirus causes severe diarrhea in children."],
        
        "Norovirus":  ["Norovirus is a contagious stomach flu."],
        
        "Enteric Fever":  ["Enteric Fever is a severe bacterial infection."],
        
        "H1N1 (Swine Flu)": ["H1N1 (Swine Flu) is a subtype of influenza."],
      }  
    


    
Diagnostic_Test: {
        "Influenza": ["Rapid Antigen Test", "RT-PCR"],
        "Dengue": ["NS1 Antigen", "Dengue IgM & IgG Antibody"],
        "Malaria": ["Blood Smear", "Rapid Diagnostic Test"],
        "Typhoid Fever": ["Blood Culture", "Widal Test"],
        "Pneumonia": ["Chest X-ray", "Blood Tests"],
        "Appendicitis": ["Abdominal Ultrasound", "CT Scan"],
        "Gastroenteritis": ["Stool Culture", "Blood Tests"],
        "Migraine": ["Clinical Diagnosis", "CT/MRI (if necessary)"],
        "Urinary Tract Infection (UTI)": ["Urine Culture", "Urine Dipstick Test"],
        "Bronchitis": ["Chest X-ray", "Sputum Culture"],
        "COVID-19": ["RT-PCR", "Rapid Antigen Test"],
        "Sinusitis": ["CT Scan", "Nasal Endoscopy"],
        "Chikungunya": ["Serology Test", "RT-PCR"],
        "Chickenpox": ["Clinical Diagnosis", "Blood Test (for IgM)"],
        "Zika Virus": ["RT-PCR", "Serology Test"],
        "Leptospirosis": ["Blood Tests", "Serology Test"],
        "Hepatitis A": ["Liver Function Test", "Hepatitis A IgM Antibody"],
        "Hepatitis B": ["HBV Surface Antigen", "HBV DNA PCR"],
        "Cholera": ["Stool Culture", "Rapid Diagnostic Test"],
        "Tetanus": ["Clinical Diagnosis", "Tetanus Toxin Test"],
        "Rabies": ["Saliva Test", "Serum Antibodies"],
        "Mumps": ["Serum IgM Antibody Test", "RT-PCR"],
        "Measles": ["Serum IgM Test", "RT-PCR"],
        "Diphtheria": ["Throat Culture", "Cultural Test for C. diphtheriae"],
        "Scarlet Fever": ["Throat Culture", "Rapid Antigen Test"],
        "Typhus": ["Blood Test (Serology)", "PCR"],
        "Rotavirus": ["Stool Antigen Test", "RT-PCR"],
        "Norovirus": ["Stool Sample Test", "PCR"],
        "Enteric Fever": ["Blood Culture", "Widal Test"],
        "H1N1 (Swine Flu)": ["RT-PCR", "Rapid Antigen Test"]
    }
Precautions: {
        "Influenza": ["Avoid close contact", "Cover mouth when coughing", "Get vaccinated annually"],
        "Dengue": ["Use mosquito repellent", "Wear long sleeves and pants", "Sleep under mosquito nets"],
        "Malaria": ["Use insect repellents", "Sleep under a mosquito net", "Take anti-malarial drugs as prescribed"],
        "Typhoid Fever": ["Drink clean water", "Wash hands frequently", "Avoid eating raw food"],
        "Pneumonia": ["Cover mouth when coughing", "Get vaccinated", "Avoid smoking"],
        "Appendicitis": ["Seek medical help immediately", "Avoid self-medication"],
        "Gastroenteritis": ["Wash hands regularly", "Avoid contaminated food or water"],
        "Migraine": ["Avoid triggers like bright lights and loud sounds", "Manage stress"],
        "Urinary Tract Infection (UTI)": ["Drink plenty of water", "Avoid holding urine for too long", "Wipe from front to back"],
        "Bronchitis": ["Avoid smoke and pollution", "Rest and hydrate"],
        "COVID-19": ["Wear masks", "Social distancing", "Frequent handwashing"],
        "Sinusitis": ["Avoid allergens", "Use a humidifier", "Drink plenty of fluids"],
        "Chikungunya": ["Avoid mosquito bites", "Stay indoors during peak mosquito activity"],
        "Chickenpox": ["Avoid scratching blisters", "Stay home until all blisters scab over"],
        "Zika Virus": ["Avoid mosquito bites", "Wear long sleeves", "Use insect repellent"],
        "Leptospirosis": ["Avoid exposure to contaminated water", "Wear protective gear"],
        "Hepatitis A": ["Get vaccinated", "Avoid unclean food and water"],
        "Hepatitis B": ["Get vaccinated", "Avoid sharing needles"],
        "Cholera": ["Drink clean water", "Use ORS to stay hydrated"],
        "Tetanus": ["Get vaccinated", "Avoid open wounds and cuts"],
        "Rabies": ["Avoid contact with stray animals", "Get post-exposure vaccination if bitten"],
        "Mumps": ["Isolate from others", "Rest and hydrate"],
        "Measles": ["Get vaccinated", "Avoid contact with others when sick"],
        "Diphtheria": ["Get vaccinated", "Avoid contact with infected individuals"],
        "Scarlet Fever": ["Take prescribed antibiotics", "Avoid close contact with others"],
        "Typhus": ["Wear protective clothing", "Avoid flea bites"],
        "Rotavirus": ["Practice good hygiene", "Ensure children are vaccinated"],
        "Norovirus": ["Wash hands regularly", "Avoid contaminated food"],
        "Enteric Fever": ["Drink clean water", "Wash hands before eating"],
        "H1N1 (Swine Flu)": ["Get vaccinated", "Avoid close contact with infected individuals"]
    }



data = []
for i in range(1000):
    idx = random.randint(0, len(diseases) - 1)
    disease_name = diseases[idx]

    for symptom in symptoms[disease_name]:
        data.append({
            "diseases": disease_name,
            "symptoms": symptom,
            "medicines": ', '.join(medicines[disease_name]),  # Join medicines with commas
            "description": description[disease_name][0],  # Assuming first description
            "diagnostic_Tests": ', '.join(diagnostic_Test[disease_name]),  # Join tests with commas
            "precautions": ', '.join(precautions[disease_name])  # Join precautions with commas
        })

# Create a DataFrame from your dataset
df = pd.DataFrame(data)

# Save the entire dataset to a CSV file
df.to_csv('full_dataset.csv', index=False)


# Create DataFrame
df = pd.DataFrame(data)
print(df)


# Preprocessing: Vectorizing Symptoms (Text data)
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['symptoms'])

# Encode target variables (Disease, Medicines, Description, etc.)
label_encoder_disease = LabelEncoder()
df['Encoded_disease'] = label_encoder_disease.fit_transform(df['diseases'])

# You can do similar encoding for Medicines, Description, Diagnostic Test, Precautions
# Assuming they are categorical or have a limited set of options

label_encoder_medicines = LabelEncoder()
df['Encoded_medicines'] = label_encoder_medicines.fit_transform(df['Medicines'])

# For Description and Precautions, you can use label encoding if they are categorical or TF-IDF if text-based
# If they are complex text (like paragraphs), consider using a different approach (e.g., separate vectorization for each)
# Below, I assume they are simple categories or lists of categories for simplicity

label_encoder_desc = LabelEncoder()
df['Encoded_description'] = label_encoder_desc.fit_transform(df['Description'])

label_encoder_prec = LabelEncoder()
df['Encoded_precautions'] = label_encoder_prec.fit_transform(df['Precautions'])

# Define your target variables
y_disease = df['Encoded_disease']
y_medicines = df['Encoded_medicines']
y_description = df['Encoded_description']
y_precautions = df['Encoded_precautions']

# Apply SMOTE for class imbalance on the input data and all target variables
smote = SMOTE(random_state=42)

# Resample input data and target variables
X_resampled, y_resampled_disease = smote.fit_resample(X, y_disease)

# Apply SMOTE on other target variables as well (Make sure these are encoded properly if categorical)
y_resampled_medicines = label_encoder_medicines.fit_transform(df['Medicines'])
y_resampled_description = label_encoder_desc.fit_transform(df['Description'])
y_resampled_precautions = label_encoder_prec.fit_transform(df['Precautions'])

# Ensure the sizes of X_resampled and target variables are consistent
print(f"X_resampled shape: {X_resampled.shape}")
print(f"y_resampled_disease shape: {y_resampled_disease.shape}")
print(f"y_resampled_medicines shape: {y_resampled_medicines.shape}")
print(f"y_resampled_description shape: {y_resampled_description.shape}")
print(f"y_resampled_precautions shape: {y_resampled_precautions.shape}")

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train_disease, y_test_disease = train_test_split(X_resampled, y_resampled_disease, test_size=0.2, random_state=42)
y_train_medicines, y_test_medicines = train_test_split(y_resampled_medicines, test_size=0.2, random_state=42)
y_train_description, y_test_description = train_test_split(y_resampled_description, test_size=0.2, random_state=42)
y_train_precautions, y_test_precautions = train_test_split(y_resampled_precautions, test_size=0.2, random_state=42)

# Verify that the shapes of X_train and the target variables match
print("X_train shape:", X_train.shape)
print("y_train_disease shape:", y_train_disease.shape)
print("y_train_medicines shape:", y_train_medicines.shape)
print("y_train_description shape:", y_train_description.shape)
print("y_train_precautions shape:", y_train_precautions.shape)

# Model definition
input_layer = Input(shape=(X_train.shape[1],))

# Shared dense layers
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)

# Disease output
disease_output = Dense(len(np.unique(y_disease)), activation='softmax', name='disease_output')(x)

# Medicines output
medicines_output = Dense(len(np.unique(y_medicines)), activation='softmax', name='medicines_output')(x)

# Description output
description_output = Dense(len(np.unique(y_description)), activation='softmax', name='description_output')(x)

# Precautions output
precautions_output = Dense(len(np.unique(y_precautions)), activation='softmax', name='precautions_output')(x)

# Model compilation
model = Model(inputs=input_layer, outputs=[disease_output, medicines_output, description_output, precautions_output])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'disease_output': 'sparse_categorical_crossentropy',
                    'medicines_output': 'sparse_categorical_crossentropy',
                    'description_output': 'sparse_categorical_crossentropy',
                    'precautions_output': 'sparse_categorical_crossentropy'},
              metrics=['accuracy'])

# Model summary
model.summary()

# Train the model
history = model.fit(X_train, 
                    {'disease_output': y_train_disease,
                     'medicines_output': y_train_medicines,
                     'description_output': y_train_description,
                     'precautions_output': y_train_precautions},
                    validation_data=(X_test, 
                                     {'disease_output': y_test_disease,
                                      'medicines_output': y_test_medicines,
                                      'description_output': y_test_description,
                                      'precautions_output': y_test_precautions}),
                    epochs=10,
                    batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)

# Post-processing or predictions analysis
# You can perform analysis on the predictions here
# To extract and evaluate each prediction:
y_pred_disease = tf.argmax(y_pred['disease_output'], axis=1).numpy()
y_pred_medicines = tf.argmax(y_pred['medicines_output'], axis=1).numpy()
y_pred_desc = tf.argmax(y_pred['description_output'], axis=1).numpy()
y_pred_prec = tf.argmax(y_pred['precautions_output'], axis=1).numpy()

# Accuracy and Classification Report
print("Disease Accuracy:", accuracy_score(y_test_disease, y_pred_disease))
print("Medicines Accuracy:", accuracy_score(y_test_medicines, y_pred_medicines))
print("Description Accuracy:", accuracy_score(y_test_description, y_pred_desc))
print("Precautions Accuracy:", accuracy_score(y_test_precautions, y_pred_prec))

print("\nClassification Report for Disease:\n", classification_report(y_test_disease, y_pred_disease, target_names=label_encoder_disease.classes_))

# Save the model
model.save("multi_output_disease_prediction_model.h5")

TypeError: list indices must be integers or slices, not str

In [251]:
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Your dataset
# Updated dataset with all columns
diseases: [
        "Influenza",
        "Dengue",
        "Malaria", 
        "Typhoid Fever", 
        "Pneumonia",
        "Appendicitis",
        "Gastroenteritis",
        "Migraine", 
        "Urinary Tract Infection (UTI)",
        "Bronchitis",
        "COVID-19", 
        "Sinusitis", 
        "Chikungunya",
        "Chickenpox",
        "Zika Virus",
        "Leptospirosis",
        "Hepatitis A",
        "Hepatitis B", 
        "Cholera",
        "Tetanus", 
        "Rabies",
        "Mumps", 
        "Measles",
        "Diphtheria",
        "Scarlet Fever",
        "Typhus",
        "Rotavirus", 
        "Norovirus",
        "Enteric Fever",
        "H1N1 (Swine Flu)"
    ]
symptoms: {
        "Influenza": ["Fever", "Cough", "Sore throat", "Muscle aches", "Fatigue"],
        "Dengue": ["High fever", "Severe headache", "Rash", "Muscle pain"],
        "Malaria": ["Chills", "Fever", "Fatigue", "Sweats"],
        "Typhoid Fever": ["Abdominal pain", "Loss of appetite", "High fever"],
        "Pneumonia": ["Chest pain", "Shortness of breath", "Fever", "Cough"],
        "Appendicitis": ["Sudden abdominal pain (lower right)", "Nausea", "Vomiting", "Low-grade fever"],
        "Gastroenteritis": ["Diarrhea", "Abdominal cramps", "Vomiting", "Low-grade fever"],
        "Migraine": ["Severe headache", "Sensitivity to light/sound", "Nausea"],
        "Urinary Tract Infection (UTI)": ["Painful urination", "Frequent urination", "Lower abdominal pain", "Cloudy urine"],
        "Bronchitis": ["Cough with mucus", "Wheezing", "Chest tightness"],
        "COVID-19": ["Fever", "Dry cough", "Loss of taste/smell", "Fatigue", "Difficulty breathing"],
        "Sinusitis": ["Facial pain", "Nasal congestion", "Headache", "Post-nasal drip"],
        "Chikungunya": ["High fever", "Joint pain", "Rash", "Headache"],
        "Chickenpox": ["Fever", "Rash (blisters)", "Itching", "Fatigue"],
        "Zika Virus": ["Fever", "Rash", "Conjunctivitis", "Joint pain"],
        "Leptospirosis": ["Fever", "Headache", "Muscle pain", "Abdominal pain"],
        "Hepatitis A": ["Fever", "Nausea", "Vomiting", "Jaundice", "Abdominal pain"],
        "Hepatitis B": ["Fatigue", "Jaundice", "Abdominal pain", "Nausea"],
        "Cholera": ["Severe diarrhea", "Vomiting", "Dehydration"],
        "Tetanus": ["Muscle stiffness", "Lockjaw", "Difficulty swallowing"],
        "Rabies": ["Fever", "Headache", "Hydrophobia", "Agitation"],
        "Mumps": ["Fever", "Swelling of salivary glands", "Headache"],
        "Measles": ["Fever", "Cough", "Rash", "Conjunctivitis"],
        "Diphtheria": ["Sore throat", "Fever", "Weakness", "Swollen glands"],
        "Scarlet Fever": ["Sore throat", "Red rash", "Fever", "Strawberry tongue"],
        "Typhus": ["Fever", "Rash", "Headache", "Muscle pain"],
        "Rotavirus": ["Severe diarrhea", "Vomiting", "Fever", "Abdominal cramps"],
        "Norovirus": ["Diarrhea", "Vomiting", "Stomach cramps"],
        "Enteric Fever": ["Abdominal pain", "High fever", "Diarrhea", "Weakness"],
        "H1N1 (Swine Flu)": ["Fever", "Cough", "Sore throat", "Body aches"]
    }
Medicines: {
        "Influenza": ["Paracetamol 500mg", "Ibuprofen 400mg", "Oseltamivir 75mg"],
        "Dengue": ["Acetaminophen 500mg", "ORS (Oral Rehydration Solution)"],
        "Malaria": ["Artemisinin-based Combination Therapy (ACT)"],
        "Typhoid Fever": ["Ciprofloxacin 500mg", "Azithromycin 250mg"],
        "Pneumonia": ["Amoxicillin 500mg", "Doxycycline 100mg"],
        "Appendicitis": ["Painkillers (e.g., Ibuprofen 200mg)", "Antibiotics (e.g., Ceftriaxone 1g)"],
        "Gastroenteritis": ["ORS (Oral Rehydration Solution)", "Loperamide 2mg", "Zinc 20mg"],
        "Migraine": ["Sumatriptan 50mg", "Ibuprofen 400mg"],
        "Urinary Tract Infection (UTI)": ["Nitrofurantoin 100mg", "Ciprofloxacin 500mg"],
        "Bronchitis": ["Bronchodilators (e.g., Salbutamol 2.5mg)", "Cough Suppressants (e.g., Dextromethorphan 10mg)"],
        "COVID-19": ["Paracetamol 500mg", "Favipiravir 200mg"],
        "Sinusitis": ["Decongestants (e.g., Pseudoephedrine 60mg)", "Antihistamines (e.g., Loratadine 10mg)"],
        "Chikungunya": ["Paracetamol 500mg", "Ibuprofen 200mg"],
        "Chickenpox": ["Calamine Lotion", "Antihistamines (e.g., Diphenhydramine 25mg)"],
        "Zika Virus": ["Paracetamol 500mg", "Fluids"],
        "Leptospirosis": ["Doxycycline 100mg", "Penicillin 500mg"],
        "Hepatitis A": ["Supportive care", "Hydration"],
        "Hepatitis B": ["Tenofovir 300mg", "Entecavir 0.5mg"],
        "Cholera": ["Rehydration (e.g., ORS)", "Antibiotics (e.g., Doxycycline 300mg)"],
        "Tetanus": ["Tetanus vaccination", "Painkillers (e.g., Ibuprofen 400mg)"],
        "Rabies": ["Post-exposure prophylaxis (PEP) with Rabies Vaccine"],
        "Mumps": ["Supportive care", "Pain relief (e.g., Paracetamol 500mg)"],
        "Measles": ["Vitamin A 100,000 IU (for children)", "Supportive care"],
        "Diphtheria": ["Penicillin 250mg", "Diphtheria Toxoid Vaccine"],
        "Scarlet Fever": ["Penicillin 500mg", "Amoxicillin 500mg"],
        "Typhus": ["Doxycycline 100mg", "Chloramphenicol 500mg"],
        "Rotavirus": ["ORS (Oral Rehydration Solution)", "Zinc 20mg"],
        "Norovirus": ["Fluids", "Antiemetics (e.g., Ondansetron 4mg)"],
        "Enteric Fever": ["Ciprofloxacin 500mg", "Azithromycin 250mg"],
        "H1N1 (Swine Flu)": ["Oseltamivir 75mg", "Paracetamol 500mg"]
    }

description:{
        "Influenza": ["The disease Influenza causes various symptoms and can be diagnosed through specific tests."],
        
        "Dengue": ["Dengue causes a high fever with severe headache and rash."],
        
        "Malaria": ["Malaria involves chills, fever, and sweats."],
        
        "Typhoid Fever":  ["Typhoid is a bacterial infection that requires antibiotics."],
        
        "Pneumonia":["Pneumonia causes breathing difficulties and chest pain."],
        
        "Appendicitis": ["Appendicitis needs immediate surgery to avoid complications."],
        
        "Gastroenteritis": ["Gastroenteritis involves inflammation of the stomach and intestines."],
        
        "Migraine":  ["Migraines are severe headaches often accompanied by nausea."],
        
        "Urinary Tract Infection (UTI)":  ["UTI is characterized by painful urination and frequent trips to the bathroom."],
        
        "Bronchitis": ["Bronchitis causes persistent coughing and mucus production."],
        
        "COVID-19":  ["COVID-19 is a respiratory disease with diverse symptoms."],
        
        "Sinusitis": ["Sinusitis affects nasal passages and causes severe headaches."],
        
        "Chikungunya":  ["Chikungunya causes joint pain and fever."],
        
        "Chickenpox":["Chickenpox is a contagious rash illness."],
        
        "Zika Virus":  ["Zika Virus can cause birth defects in pregnant women."],
        
        "Leptospirosis": ["Leptospirosis is a bacterial infection that spreads through animal urine."],
        
        "Hepatitis A":  ["Hepatitis A is a liver infection caused by contaminated food or water."],
        
        "Hepatitis B":  ["Hepatitis B is a liver infection spread through body fluids."],
        
        "Cholera":  ["Cholera is caused by contaminated water and causes severe dehydration."],
        
        "Tetanus": ["Tetanus affects the nervous system and causes muscle stiffness."],
        
        "Rabies":  ["Rabies is a viral infection often fatal once symptoms appear."],
        
        "Mumps":  ["Mumps is a viral infection causing swollen salivary glands."],
        
        "Measles":  ["Measles causes a characteristic rash and fever."],
        
        "Diphtheria": ["Diphtheria affects the throat and nose."],
        
        "Scarlet Fever": ["Scarlet Fever involves a bright red rash."],
        
        "Typhus":  ["Typhus is transmitted by lice and fleas."],
        
        "Rotavirus":  ["Rotavirus causes severe diarrhea in children."],
        
        "Norovirus":  ["Norovirus is a contagious stomach flu."],
        
        "Enteric Fever":  ["Enteric Fever is a severe bacterial infection."],
        
        "H1N1 (Swine Flu)": ["H1N1 (Swine Flu) is a subtype of influenza."],
      }  
    


    
Diagnostic_Test: {
        "Influenza": ["Rapid Antigen Test", "RT-PCR"],
        "Dengue": ["NS1 Antigen", "Dengue IgM & IgG Antibody"],
        "Malaria": ["Blood Smear", "Rapid Diagnostic Test"],
        "Typhoid Fever": ["Blood Culture", "Widal Test"],
        "Pneumonia": ["Chest X-ray", "Blood Tests"],
        "Appendicitis": ["Abdominal Ultrasound", "CT Scan"],
        "Gastroenteritis": ["Stool Culture", "Blood Tests"],
        "Migraine": ["Clinical Diagnosis", "CT/MRI (if necessary)"],
        "Urinary Tract Infection (UTI)": ["Urine Culture", "Urine Dipstick Test"],
        "Bronchitis": ["Chest X-ray", "Sputum Culture"],
        "COVID-19": ["RT-PCR", "Rapid Antigen Test"],
        "Sinusitis": ["CT Scan", "Nasal Endoscopy"],
        "Chikungunya": ["Serology Test", "RT-PCR"],
        "Chickenpox": ["Clinical Diagnosis", "Blood Test (for IgM)"],
        "Zika Virus": ["RT-PCR", "Serology Test"],
        "Leptospirosis": ["Blood Tests", "Serology Test"],
        "Hepatitis A": ["Liver Function Test", "Hepatitis A IgM Antibody"],
        "Hepatitis B": ["HBV Surface Antigen", "HBV DNA PCR"],
        "Cholera": ["Stool Culture", "Rapid Diagnostic Test"],
        "Tetanus": ["Clinical Diagnosis", "Tetanus Toxin Test"],
        "Rabies": ["Saliva Test", "Serum Antibodies"],
        "Mumps": ["Serum IgM Antibody Test", "RT-PCR"],
        "Measles": ["Serum IgM Test", "RT-PCR"],
        "Diphtheria": ["Throat Culture", "Cultural Test for C. diphtheriae"],
        "Scarlet Fever": ["Throat Culture", "Rapid Antigen Test"],
        "Typhus": ["Blood Test (Serology)", "PCR"],
        "Rotavirus": ["Stool Antigen Test", "RT-PCR"],
        "Norovirus": ["Stool Sample Test", "PCR"],
        "Enteric Fever": ["Blood Culture", "Widal Test"],
        "H1N1 (Swine Flu)": ["RT-PCR", "Rapid Antigen Test"]
    }
Precautions: {
        "Influenza": ["Avoid close contact", "Cover mouth when coughing", "Get vaccinated annually"],
        "Dengue": ["Use mosquito repellent", "Wear long sleeves and pants", "Sleep under mosquito nets"],
        "Malaria": ["Use insect repellents", "Sleep under a mosquito net", "Take anti-malarial drugs as prescribed"],
        "Typhoid Fever": ["Drink clean water", "Wash hands frequently", "Avoid eating raw food"],
        "Pneumonia": ["Cover mouth when coughing", "Get vaccinated", "Avoid smoking"],
        "Appendicitis": ["Seek medical help immediately", "Avoid self-medication"],
        "Gastroenteritis": ["Wash hands regularly", "Avoid contaminated food or water"],
        "Migraine": ["Avoid triggers like bright lights and loud sounds", "Manage stress"],
        "Urinary Tract Infection (UTI)": ["Drink plenty of water", "Avoid holding urine for too long", "Wipe from front to back"],
        "Bronchitis": ["Avoid smoke and pollution", "Rest and hydrate"],
        "COVID-19": ["Wear masks", "Social distancing", "Frequent handwashing"],
        "Sinusitis": ["Avoid allergens", "Use a humidifier", "Drink plenty of fluids"],
        "Chikungunya": ["Avoid mosquito bites", "Stay indoors during peak mosquito activity"],
        "Chickenpox": ["Avoid scratching blisters", "Stay home until all blisters scab over"],
        "Zika Virus": ["Avoid mosquito bites", "Wear long sleeves", "Use insect repellent"],
        "Leptospirosis": ["Avoid exposure to contaminated water", "Wear protective gear"],
        "Hepatitis A": ["Get vaccinated", "Avoid unclean food and water"],
        "Hepatitis B": ["Get vaccinated", "Avoid sharing needles"],
        "Cholera": ["Drink clean water", "Use ORS to stay hydrated"],
        "Tetanus": ["Get vaccinated", "Avoid open wounds and cuts"],
        "Rabies": ["Avoid contact with stray animals", "Get post-exposure vaccination if bitten"],
        "Mumps": ["Isolate from others", "Rest and hydrate"],
        "Measles": ["Get vaccinated", "Avoid contact with others when sick"],
        "Diphtheria": ["Get vaccinated", "Avoid contact with infected individuals"],
        "Scarlet Fever": ["Take prescribed antibiotics", "Avoid close contact with others"],
        "Typhus": ["Wear protective clothing", "Avoid flea bites"],
        "Rotavirus": ["Practice good hygiene", "Ensure children are vaccinated"],
        "Norovirus": ["Wash hands regularly", "Avoid contaminated food"],
        "Enteric Fever": ["Drink clean water", "Wash hands before eating"],
        "H1N1 (Swine Flu)": ["Get vaccinated", "Avoid close contact with infected individuals"]
    }



# Generate random data
data = []
for i in range(1000):  # Generate 1000 data points
    idx = random.randint(0, len(diseases) - 1)
    data.append({
        "diseases": diseases[idx],
        "symptoms": symptoms[idx],
        "Medicines": medicines[idx],
        "Description": f"The disease {diseases[idx]} causes various symptoms and can be diagnosed through specific tests.",
        "Diagnostic Tests": "Tests to diagnose disease (e.g., blood tests, X-ray, PCR)",
        "Precautions": "Rest, hydration, vaccination (when applicable)"
    })
# Create a DataFrame from your dataset
df = pd.DataFrame(data)

# Save the entire dataset to a CSV file
df.to_csv('full_dataset.csv', index=False)


# Create DataFrame
df = pd.DataFrame(data)
print(df)


# Preprocess the data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['symptoms'])

# Encode target variables
label_encoder_disease = LabelEncoder()
df['Encoded_disease'] = label_encoder_disease.fit_transform(df['diseases'])

label_encoder_medicines = LabelEncoder()
df['Encoded_medicines'] = label_encoder_medicines.fit_transform(df['Medicines'])

# ... (Encode other target variables as needed)

# Define target variables
y_disease = df['Encoded_disease']
y_medicines = df['Encoded_medicines']
# ... (Define other target variables)

# Split the data into training and testing sets
X_train, X_test, y_train_disease, y_test_disease = train_test_split(X, y_disease, test_size=0.2, random_state=42)
X_train, X_test, y_train_medicines, y_test_medicines = train_test_split(X, y_medicines, test_size=0.2, random_state=42)
# ... (Split for other target variables)

# Ensure consistent sample sizes
assert X_train.shape[0] == y_train_disease.shape[0] == y_train_medicines.shape[0]  # ... and other targets

# Define the multi-output model
input_layer = Input(shape=(X_train.shape[1],))

# Shared layers
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)

# Output layers
disease_output = Dense(len(np.unique(y_disease)), activation='softmax', name='disease_output')(x)
medicines_output = Dense(len(np.unique(y_medicines)), activation='softmax', name='medicines_output')(x)
# ... (Define other output layers as needed)

model = Model(inputs=input_layer, outputs=[disease_output, medicines_output])  # ... and other outputs

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'disease_output': 'sparse_categorical_crossentropy',
                    'medicines_output': 'sparse_categorical_crossentropy'},  # ... and other losses
              metrics=['accuracy', 'accuracy'])  # One accuracy metric for each output

# Train the model
history = model.fit(X_train,
                    {'disease_output': y_train_disease,
                     'medicines_output': y_train_medicines},  # ... and other targets
                    validation_data=(X_test,
                                     {'disease_output': y_test_disease,
                                      'medicines_output': y_test_medicines}),  # ... and other targets
                    epochs=10,
                    batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)

# Post-processing and evaluation
y_pred_disease = tf.argmax(y_pred[0], axis=1).numpy()
y_pred_medicines = tf.argmax(y_pred[1], axis=1).numpy()
# ... (Extract predictions for other outputs)

print("Disease Accuracy:", accuracy_score(y_test_disease, y_pred_disease))
print("Medicines Accuracy:", accuracy_score(y_test_medicines, y_pred_medicines))
# ... (Print accuracy for other outputs)

print("\nClassification Report for Disease:\n", classification_report(y_test_disease, y_pred_disease, target_names=label_encoder_disease.classes_))
# ... (Print classification reports for other outputs)

# Save the model
model.save("multi_output_disease_prediction_model.h5")

                          diseases  \
0                    Leptospirosis   
1                     Appendicitis   
2                       Zika Virus   
3                            Mumps   
4                           Typhus   
..                             ...   
995  Urinary Tract Infection (UTI)   
996                        Cholera   
997  Urinary Tract Infection (UTI)   
998                         Typhus   
999                      Sinusitis   

                                              symptoms  \
0         Fever, Headache, Muscle pain, Abdominal pain   
1    Sudden abdominal pain (lower right), Nausea, V...   
2              Fever, Rash, Conjunctivitis, Joint pain   
3         Fever, Swelling of salivary glands, Headache   
4                   Fever, Rash, Headache, Muscle pain   
..                                                 ...   
995  Painful urination, Frequent urination, Lower a...   
996             Severe diarrhea, Vomiting, Dehydration   
997  Painful urinatio

Disease Accuracy: 1.0
Medicines Accuracy: 1.0

Classification Report for Disease:
                                precision    recall  f1-score   support

                 Appendicitis       1.00      1.00      1.00         5
                   Bronchitis       1.00      1.00      1.00         5
                     COVID-19       1.00      1.00      1.00        10
                   Chickenpox       1.00      1.00      1.00         8
                  Chikungunya       1.00      1.00      1.00         5
                      Cholera       1.00      1.00      1.00         5
                       Dengue       1.00      1.00      1.00        10
                   Diphtheria       1.00      1.00      1.00         7
                Enteric Fever       1.00      1.00      1.00         7
              Gastroenteritis       1.00      1.00      1.00         5
             H1N1 (Swine Flu)       1.00      1.00      1.00         4
                  Hepatitis A       1.00      1.00      1.00    

In [247]:
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import numpy as np
df=pd.read_csv("full_dataset5.csv")


# Preprocess the data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['symptoms'])

# Encode target variables
label_encoder_disease = LabelEncoder()
df['Encoded_disease'] = label_encoder_disease.fit_transform(df['diseases'])

label_encoder_medicines = LabelEncoder()
df['Encoded_medicines'] = label_encoder_medicines.fit_transform(df['medicines'])

label_encoder_medicines = LabelEncoder()
df['Encoded_desciption'] = label_encoder_medicines.fit_transform(df['description'])

label_encoder_medicines = LabelEncoder()
df['Encoded_precautions'] = label_encoder_medicines.fit_transform(df['precautions'])

label_encoder_medicines = LabelEncoder()
df['Encoded_diagnostic_Tests'] = label_encoder_medicines.fit_transform(df['diagnostic_Tests'])

# ... (Encode other target variables as needed)

# Define target variables
y_disease = df['Encoded_disease']
y_medicines = df['Encoded_medicines']
y_disease = df['Encoded_desciption']
y_medicines = df['Encoded_precautions']
y_medicines = df['Encoded_diagnostic_Tests']
# ... (Define other target variables)

# Split the data into training and testing sets
X_train, X_test, y_train_disease, y_test_disease = train_test_split(X, y_disease, test_size=0.2, random_state=42)
X_train, X_test, y_train_medicines, y_test_medicines = train_test_split(X, y_medicines, test_size=0.2, random_state=42)
X_train, X_test, y_train_desciption, y_test_desciption = train_test_split(X, y_description, test_size=0.2, random_state=42)
X_train, X_test, y_train_precautions, y_test_precautions = train_test_split(X, y_precautions, test_size=0.2, random_state=42)
X_train, X_test, y_train_diagnostic_Tests, y_test_diagnostic_Tests = train_test_split(X, y_diagnostic_Tests, test_size=0.2, random_state=42)
# ... (Split for other target variables)

# Ensure consistent sample sizes
'''assert X_train.shape[0] == y_train_disease.shape[0] == y_train_medicines.shape[0]  # ... and other targets
assert X_train.shape[0] == y_train_disease.shape[0] == y_train_medicines.shape[0] 
assert X_train.shape[0] == y_train_disease.shape[0] == y_train_medicines.shape[0] 
assert X_train.shape[0] == y_train_disease.shape[0] == y_train_medicines.shape[0] '''
# Define the multi-output model
input_layer = Input(shape=(X_train.shape[1],))

# Shared layers
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)

# Output layers
disease_output = Dense(len(np.unique(y_disease)), activation='softmax', name='disease_output')(x)
medicines_output = Dense(len(np.unique(y_medicines)), activation='softmax', name='medicines_output')(x)
description_output = Dense(len(np.unique(y_description)), activation='softmax', name='description_output')(x)
precautions_output = Dense(len(np.unique(y_precautions)), activation='softmax', name='precautions_output')(x)
diagnostic_Tests_output = Dense(len(np.unique(y_diagnostic_Tests)), activation='softmax', name='diagnostic_Tests_output')(x)
# ... (Define other output layers as needed)

model = Model(inputs=input_layer, outputs=[disease_output, medicines_output,description_output,precautions_output,diagnostic_Tests_output])  # ... and other outputs

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'disease_output': 'sparse_categorical_crossentropy',
                    'medicines_output': 'sparse_categorical_crossentropy',
                    'description_output': 'sparse_categorical_crossentropy',
                    'precautions_output':'sparse_categorical_crossentropy',
                    'diagnostic_Tests_output':'sparse_categorical_crossentropy'},  # ... and other losses
              metrics=['accuracy', 'accuracy','accuracy', 'accuracy', 'accuracy'])  # One accuracy metric for each output

# Train the model
history = model.fit(X_train,
                    {'disease_output': y_train_disease,
                     'medicines_output': y_train_medicines,
                      'description_output':y_train_description,
                     'precautions_output':y_train_precautions,
                     'diagnostic_Tests_output':y_train_diagnostic_Tests},  # ... and other targets
                    validation_data=(X_test,
                                     {'disease_output': y_test_disease,
                                      'medicines_output': y_test_medicines,
                                     'description_output':y_test_description,
                     'precautions_output':y_test_precautions,
                     'diagnostic_Tests_output':y_test_diagnostic_Tests}),  # ... and other targets
                    epochs=10,
                    batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)

# Post-processing and evaluation
y_pred_disease = tf.argmax(y_pred[0], axis=1).numpy()
y_pred_medicines = tf.argmax(y_pred[1], axis=1).numpy()
y_pred_description = tf.argmax(y_pred[0], axis=1).numpy()
y_pred_precautions = tf.argmax(y_pred[1], axis=1).numpy()
y_pred_diagnostic_Tests = tf.argmax(y_pred[1], axis=1).numpy()
# ... (Extract predictions for other outputs)
# ... (Extract predictions for other outputs)

print("Disease Accuracy:", accuracy_score(y_test_disease, y_pred_disease))
print("Medicines Accuracy:", accuracy_score(y_test_medicines, y_pred_medicines))
print("Desciption Accuracy:", accuracy_score(y_test_description, y_pred_description))
print("Medicines Accuracy:", accuracy_score(y_test_precautions, y_pred_precautions))
print("Medicines Accuracy:", accuracy_score(y_test_diagnostic_Tests, y_pred_diagnostic_Tests))
# ... (Print accuracy for other outputs)

print("\nClassification Report for Disease:\n", classification_report(y_test_disease, y_pred_disease, target_names=label_encoder_disease.classes_))
# ... (Print classification reports for other outputs)

# Save the model
model.save("multi_output_disease_prediction_model.h5")

ValueError: Found input variables with inconsistent numbers of samples: [3898, 1000]

In [256]:
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import numpy as np

df=pd.read_csv("full_dataset5.csv")


# Preprocess the data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['symptoms'])

# Encode target variables
label_encoder_disease = LabelEncoder()
df['Encoded_disease'] = label_encoder_disease.fit_transform(df['diseases'])

label_encoder_medicines = LabelEncoder()
df['Encoded_medicines'] = label_encoder_medicines.fit_transform(df['medicines'])

# ... (Encode other target variables as needed)

# Define target variables
y_disease = df['Encoded_disease']
y_medicines = df['Encoded_medicines']
# ... (Define other target variables)

# Split the data into training and testing sets
X_train, X_test, y_train_disease, y_test_disease = train_test_split(X, y_disease, test_size=0.2, random_state=42)
X_train, X_test, y_train_medicines, y_test_medicines = train_test_split(X, y_medicines, test_size=0.2, random_state=42)
# ... (Split for other target variables)

# Ensure consistent sample sizes
assert X_train.shape[0] == y_train_disease.shape[0] == y_train_medicines.shape[0]  # ... and other targets

# Define the multi-output model
input_layer = Input(shape=(X_train.shape[1],))

# Shared layers
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)

# Output layers
disease_output = Dense(len(np.unique(y_disease)), activation='softmax', name='disease_output')(x)
medicines_output = Dense(len(np.unique(y_medicines)), activation='softmax', name='medicines_output')(x)
# ... (Define other output layers as needed)

model = Model(inputs=input_layer, outputs=[disease_output, medicines_output])  # ... and other outputs

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'disease_output': 'sparse_categorical_crossentropy',
                    'medicines_output': 'sparse_categorical_crossentropy'},  # ... and other losses
              metrics=['accuracy', 'accuracy'])  # One accuracy metric for each output

# Train the model
history = model.fit(X_train,
                    {'disease_output': y_train_disease,
                     'medicines_output': y_train_medicines},  # ... and other targets
                    validation_data=(X_test,
                                     {'disease_output': y_test_disease,
                                      'medicines_output': y_test_medicines}),  # ... and other targets
                    epochs=10,
                    batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)

# Post-processing and evaluation
y_pred_disease = tf.argmax(y_pred[0], axis=1).numpy()
y_pred_medicines = tf.argmax(y_pred[1], axis=1).numpy()
# ... (Extract predictions for other outputs)

print("Disease Accuracy:", accuracy_score(y_test_disease, y_pred_disease))
print("Medicines Accuracy:", accuracy_score(y_test_medicines, y_pred_medicines))
# ... (Print accuracy for other outputs)

print("\nClassification Report for Disease:\n", classification_report(y_test_disease, y_pred_disease, target_names=label_encoder_disease.classes_))
# ... (Print classification reports for other outputs)

# Save the model
model.save("multi_output_disease_prediction_model.h5")

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - disease_output_accuracy: 0.0617 - disease_output_loss: 3.3745 - loss: 6.7087 - medicines_output_accuracy: 0.1032 - medicines_output_loss: 3.3342 - val_disease_output_accuracy: 0.1487 - val_disease_output_loss: 3.1973 - val_loss: 6.3310 - val_medicines_output_accuracy: 0.2090 - val_medicines_output_loss: 3.1347
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - disease_output_accuracy: 0.1707 - disease_output_loss: 3.0714 - loss: 6.0612 - medicines_output_accuracy: 0.2154 - medicines_output_loss: 2.9898 - val_disease_output_accuracy: 0.3577 - val_disease_output_loss: 2.5505 - val_loss: 5.0007 - val_medicines_output_accuracy: 0.3256 - val_medicines_output_loss: 2.4559
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - disease_output_accuracy: 0.4020 - disease_output_loss: 2.3263 - loss: 4.5450 - medicines_output_accuracy: 0.3787 - medicines_output_loss: 2.2186 - val_disease_output_accuracy: 0.4744 - val_disease_output_loss: 1.7756 - v

F:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
F:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
F:\anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Disease Accuracy: 0.49230769230769234
Medicines Accuracy: 0.5064102564102564

Classification Report for Disease:
                                precision    recall  f1-score   support

                 Appendicitis       0.28      0.43      0.34        21
                   Bronchitis       1.00      1.00      1.00        20
                     COVID-19       1.00      0.51      0.68        39
                   Chickenpox       1.00      0.52      0.69        21
                  Chikungunya       0.62      0.50      0.55        16
                      Cholera       0.43      0.69      0.53        13
                       Dengue       0.70      0.29      0.41        24
                   Diphtheria       1.00      0.27      0.42        26
                Enteric Fever       0.41      1.00      0.58        44
              Gastroenteritis       0.32      0.71      0.44        28
             H1N1 (Swine Flu)       1.00      0.32      0.49        28
                  Hepatitis A    